# Initialize

In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2

import os, sys, logging, datetime, warnings, shutil
from importlib import reload

import numpy as np
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')   # avoid dark backgrounds from dark themed vscode

import kalepy as kale
import kalepy.utils
import kalepy.plot
# The `nbshow` command runs `plt.show()` in interactive jupyter notebooks, but closes
#   figures when run from the command-line (notebooks are converted to scripts and run as tests)
from kalepy.plot import nbshow

import warnings
# warnings.simplefilter("error")   # WARNING: this is breaking jupyter at the moment (2021-02-14)

In [ ]:
np.random.seed(1234567)

In [ ]:
def draw_example(xx, yy, data):
    fig, axes = plt.subplots(figsize=[12, 4], ncols=2)

    for ii, ax in enumerate(axes):
        interp = bool(ii)
        vals = kale.sample.sample_grid([xx, yy], data, 20000, interpolate=interp)
        try:
            ax.set_title(f"{interp=}")
            ax.scatter(*vals, color='pink', s=3, zorder=20, alpha=0.3)
            ax.hist2d(*vals, bins=4)
        except:
            print(f"{vals=}")
            raise

    return fig, vals

# Test 0 - 1D Distribution

In [ ]:
import sympy as sym

x, x1, x2, dx, dy, y1, F = sym.symbols('x x1 x2 dx dy y1 F')
func = (dy/dx) * (x - x1) + y1

cdf = sym.integrate(func, (x, x1, x)).simplify()
print(f"{cdf=}")

y1sol = sym.solve(cdf.subs(x, x2) - 1, y1)[0]
print(f"{y1sol=}")
# print(f"{y1sol.subs(x2 - x1, dx)=}")

sol = sym.solve(cdf - F, x)
sol = sol[1]
print(f"x(F) = {sol}")
# print(f"x(F) = {sol.subs(x2 - x1, dx)}")
# print(f"x(F) = {sol.subs(y1, y1sol).subs(x2 - x1, dx).simplify()}")

print("----")
print(cdf.subs(y1, y1sol).subs(x, x1).simplify())
print(cdf.subs(y1, y1sol).subs(x, x2).simplify())

In [ ]:
def _uniform_locs(num, random=True):
    if random:
        xx = np.random.uniform(0.0, 1.0, size=num)
    else:
        xx = np.linspace(0.0, 1.0, num)
    return xx

'''
def _sample_interval(x1, x2, dy, locs):
    dx = x2 - x1
    vals = locs.copy()
    sel = ~np.isclose(dy, 0.0)
    if not np.any(sel):
        return vals
    x1 = x1[sel]
    x2 = x2[sel]
    dy = dy[sel]
    ll = locs[sel]

    y1 = (dx + dy*x1*x2 - (dy/2)*(x1**2 + x2**2)) / dx**2
    # area = 0.5 * (2*y1 + dy) * dx
    # print(f"{x1=:+5.2f} {x2=:+5.2f} {dy=:+5.2f} {y1=:+5.2f} {area=:+5.2f} m={dy/dx:+.2e}")
    vals[sel] = dx * (2*ll*dy + dx * y1**2)
    vals[sel] = (dy*x1 - dx*y1 + np.sqrt(vals[sel])) / dy
    bl = (vals < x1) & ~np.isclose(vals, x1)
    br = (vals > x2) & ~np.isclose(vals, x2)
    if np.any(bl | br):
        raise
    return vals
'''

def _sample_interval(x1, x2, dy, locs):
    return kale.sample._intrabin_linear_interp(locs, dy)

def sample_distribution_interpolated(xx, yy, num=1e5):
    nbins = len(xx) - 1    
    npb = kale.utils.trapz_dens_to_mass(yy, xx)
    npb = npb / npb.sum()
    assert len(xx) == len(yy)
    assert npb.size == nbins
    assert npb.sum() == 1.0
    npb *= num
    npb = np.around(npb).astype(int)
    temp = npb[-1]
    npb[-1] = int(num) - (npb.sum() - temp)
    bw = xx.max() - xx.min()

    # yy = yy * 2
    normalizations = kale.utils.trapz_dens_to_mass(yy, xx)

    zz = []
    for ii, nn in enumerate(npb):
        tt = _uniform_locs(nn)
        dx = xx[ii+1] - xx[ii]
        ys = yy[ii:ii+2].copy()
        ys = ys / normalizations[ii]
        # dy = yy[ii+1] - yy[ii]
        dy = np.diff(ys)

        # norm = kale.utils.trapz_dens_to_mass(yy[ii:ii+2], xx[ii:ii+2])[0]
        # print(f"{ii=}, {norm=}, {dx=}, {dy=}")
        # norm = normalizations[ii]
        norm = 1.0

        # tt = _sample_interval(xx[ii], xx[ii+1], dy, tt)
        print(f"{kale.utils.stats_str(tt)}")
        # tt = _sample_interval(0.0, 1.0, dy * dx / norm, tt)
        tt = _sample_interval(np.zeros_like(tt), np.ones_like(tt), np.ones_like(tt) * dy * dx / norm, tt)
        print(f"{xx[ii]=} {dx=}")
        print(f"{kale.utils.stats_str(tt)}")
        tt = xx[ii] + dx * tt
        
        zz = np.concatenate([zz, tt])
    
    return zz

def plot_samples(ax, xx, yy, zz, bins):
    xc = kale.utils.midpoints(bins)
    yc = np.interp(xc, xx, yy)
    cc, = ax.plot([], [], ls='--')
    cc = cc.get_color()
    kale.plot.draw_hist1d(ax, bins, yc, color=cc, ls='--', alpha=0.5, lw=3.0)

    ye = np.interp(bins, xx, yy)
    mass = kale.utils.trapz_dens_to_mass(ye, bins)
    assert mass.sum() == 1.0
    nums = np.around(mass * zz.size)
    if nums.sum() > 0:
        errs = np.sqrt(nums) / nums.sum() / np.diff(bins)
    else:
        errs = np.zeros_like(nums)
    # print(f"{errs=} {nums=}")
    ax.errorbar(xc, yc, yerr=2*errs, color=cc, ls='none')

    nn, ee, _ = ax.hist(zz, bins, histtype='step', alpha=0.75, color=cc, density=True, lw=1.5)
       
    return

xx = [0.0, 1.0, 2.0]
# xx = [0.0, 0.5, 1.0]
# yy = [0.0, 4.0, 2.0]
yy = [0.0, 4.0, 4.0]
# xx = [-3.0, 4.0]
# yy = [2.0, 4.0]
# xx = [-3.0, 0.0, 3.0]
# yy = [0.0, 4.0, 8.0]

NUM = int(1e4)
xx = [-2.0, -1.0, 1.0]
yy = [3.0, 4.0, 0.0]
xx = np.asarray(xx)
yy = np.asarray(yy)
norm = kale.utils.trapz_nd(yy, xx)
yy = yy / norm
print(f"{yy=}")

zz = yy * 100

plt.plot(xx, yy)
ax = plt.gca()
x1 = xx
x2 = kale.utils.subdivide(xx, 7)
plt.hist(zz, bins=x1, density=True, histtype='step')
plt.hist(zz, bins=x2, density=True, histtype='step')

zz = kale.sample.sample_grid(xx, yy, NUM, interpolate=True).squeeze()
plt.scatter(zz, np.zeros_like(zz))
plt.hist(zz, bins=x1, density=True, histtype='step', ls='--')
plt.hist(zz, bins=x2, density=True, histtype='step', ls='--')

bins = xx.copy()
ax.plot(xx, yy, 'k--', lw=3.0, alpha=0.5)
plot_samples(ax, xx, yy, zz, bins)

bins = kale.utils.subdivide(bins, num=3)
plot_samples(ax, xx, yy, zz, bins)

# bins = kale.utils.subdivide(bins, num=1)
# plot_samples(ax, xx, yy, ss, bins)

nbshow()

In [ ]:
ss = kale.sample_grid(xx, yy, nsamp=100000)[0]
print(ss.shape)

fig, ax = plt.subplots(figsize=[8, 3])
plt.tight_layout()

bins = xx.copy()
ax.plot(xx, yy, 'k--', lw=3.0, alpha=0.5)
plot_samples(ax, xx, yy, ss, bins)

bins = kale.utils.subdivide(bins, num=3)
plot_samples(ax, xx, yy, ss, bins)

# bins = kale.utils.subdivide(bins, num=1)
# plot_samples(ax, xx, yy, ss, bins)

nbshow()

In [ ]:
# xx = np.linspace(-2.0, 2.0, 10)
# xx = np.linspace(0.0, 2.0, 5)
# xx = np.arange(10.0)
xx = np.arange(6.0, 12.0)
yy = xx ** 2 + 0.5 + 10*xx
zz = yy / kale.utils.trapz_nd(yy, xx)

fig, axes = plt.subplots(figsize=[12, 4], ncols=2)

for ii, ax in enumerate(axes):
    interp = bool(ii)
    vals = kale.sample.sample_grid(xx, yy, 1e5, interpolate=interp).squeeze()
    
    ax.set_title(f"{interp=}")
    ax.plot(xx, zz, 'k--')

    kale.plot.hist1d(vals, xx, ax, density=True, probability=True)
    div = kale.utils.subdivide(xx, 3)
    kale.plot.hist1d(vals, div, ax, density=True, probability=True)

nbshow()

In [ ]:
def draw_example_2d(edges, data, num=1e4):
    # xx, yy, data = [np.asarray(vv) for vv in [xx, yy, data]]
    # edges = [xx, yy]
    xx, yy = edges
    zz = data / kale.utils.trapz_nd(data, edges)

    # -----------------------------

    fig = plt.figure(figsize=[10, 4])
    gs_base = fig.add_gridspec(1, 2, hspace=0.25, wspace=0.25)
    gs = np.empty((2, 2), dtype=object)
    axes = np.empty((2, 2, 2, 2), dtype=object)
    cmap = 'RdBu'
    norm = None
    for jj in range(2):
        interp = bool(jj)
        samples = kale.sample_grid(edges, data, nsamp=num, interpolate=interp)    
        
        for ii in range(1):            
            gs[ii, jj] = gs_base[ii, jj].subgridspec(2, 2)
            for aa, bb in np.ndindex(gs.shape):
                if aa == 0 and bb == 1:
                    continue
                kw = {}
                if aa == 1 and bb == 0:
                    kw['sharex'] = axes[ii, jj, 0, 0]
                elif aa == 1 and bb == 1:
                    kw['sharey'] = axes[ii, jj, 1, 0]
                    
                axes[ii, jj, aa, bb] = fig.add_subplot(gs[ii, jj][aa, bb], **kw)

            subdiv = jj

            ax = axes[ii, jj, 1, 0]
            # grid = np.meshgrid(xx, yy, indexing='ij')
            # ax.pcolormesh(*grid, zz, shading='auto', cmap='RdBu')
            subdiv = [kale.utils.subdivide(ee, 1) for ee in edges]
            *_, smap = ax.hist2d(*samples, bins=subdiv, cmap='RdBu', norm=norm)
            if norm is None:
                cmap = smap.get_cmap()
                norm = smap.norm
            ax.scatter(*samples, facecolor='r', edgecolor='w', alpha=5.0 / np.sqrt(num), s=2, zorder=10)

            ave = np.sum(data, axis=1)
            ave = ave / kale.utils.trapz_nd(ave, xx)
            ax = axes[ii, jj, 0, 0]
            ax.plot(xx, ave, 'k--')

            bins = xx
            kw = dict(histtype='step', alpha=0.5, density=True)
            ax.hist(samples[0], bins=bins, **kw)
            bins = kale.utils.subdivide(bins, 1)
            ax.hist(samples[0], bins=bins, **kw)

            ave = np.sum(data, axis=0)
            ave = ave / kale.utils.trapz_nd(ave, yy)
            ax = axes[ii, jj, 1, 1]
            ax.plot(ave, yy, 'k--')

            bins = yy
            kw['orientation'] = 'horizontal'
            ax.hist(samples[1], bins=bins, **kw)
            bins = kale.utils.subdivide(bins, 1)
            ax.hist(samples[1], bins=bins, **kw)

    return fig

In [ ]:
xx = [0.0, 1.0]
yy = [2.0, 4.0]
data = [
    [0.0, 1.0],
    [1.0, 2.0],
]
draw_example_2d([xx, yy], data, num=1e4)
nbshow()

In [ ]:
xx = [0.0, 1.0, 2.0]
yy = [2.0, 4.0]
data = [
    [0.0, 1.0],
    [1.0, 2.0],
    [2.0, 3.0],
]
draw_example_2d([xx, yy], data, num=1e4)
nbshow()

In [ ]:
xx = [0.0, 1.0, 2.0]
yy = [2.0, 4.0]
data = [
    [1.0, 1.0],
    [2.0, 2.0],
    [3.0, 3.0],
]
draw_example_2d([xx, yy], data, num=1e3)
nbshow()

In [ ]:
xx = [0.0, 1.0, 2.0]
yy = [2.0, 4.0]
data = [
    [1.0, 3.0],
    [2.0, 2.0],
    [3.0, 1.0],
]
draw_example_2d([xx, yy], data, num=1e4)
nbshow()

# Test 1

In [ ]:
xx = [0.0, 1.0, 2.0]  # , 3.0]
yy = [100.0, 200.0, 300.0]
# yy = np.array(yy) / 100.0

edge = [
    [0.0, 10.0, 0.0],
    # [10.0, 20.0, 10.0],
    [10.0, 20.0, 10.0],
    [0.0, 10.0, 0.0],
]

fig, vals = draw_example(xx, yy, edge)

nbshow()

## Compare 1/4ths

In [ ]:
# ---- Make sure quadrants are evenly sampled
quads, *_ = np.histogram2d(*vals, bins=(xx, yy))

tot = quads.sum()
# fraction in each quadrant
frac = quads / tot
print("frac = ", frac.flatten())
# expected poisson error
err = 1.0 / np.sqrt(tot / 4.0)
err *= 3.0   # padding
extr = 0.25 * np.array([1.0 - err, 1.0 + err])
print("exp = ", 1.0/4.0, " +- ", err, "==>", extr)
# Make sure each quadrant is roughly equal
if np.any((frac < extr[0]) | (frac > extr[1])):
    raise ValueError("Uneven distribution of points!")

## Compare 1/16ths

In [ ]:
edges = [np.sort(np.concatenate([kale.utils.midpoints(ee), ee])) for ee in [xx, yy]]
octs, *_ = np.histogram2d(*vals, bins=edges)

tot = octs.sum()
frac = octs / tot
ave = tot / octs.size
err = np.sqrt(ave)
extr = [ave - err, ave + err]
extr = np.array(extr) / tot
print(f"{ave=}, {err=}, {extr=}")

print("frac=\n", frac)
corners = np.ix_([0, 3], [0, 3])
corners = frac[corners].flatten()
print("corners=\n", corners)

centers = np.ix_([1, 2], [1, 2])
centers = frac[centers].flatten()
print("centers=\n", centers)

diags_h = np.ix_([1, 2], [0, 3])
diags_h = frac[diags_h]
diags_v = np.ix_([0, 3], [1, 2])
diags_v = frac[diags_v]
diags = np.concatenate([diags_h, diags_v]).flatten()
print("diags=\n", diags)

if not np.all(diags[:, np.newaxis] < centers[np.newaxis, :]):
    err = "diagonal values are not all smaller than center values!"
    raise ValueError(err)
    
if not np.all(corners[:, np.newaxis] < diags[np.newaxis, :]):
    err = "corner values are not all smaller than diagonal values!"
    raise ValueError(err)
    
if np.any((diags > 0.07) | (diags < 0.03)):
    raise ValueError("diagonals are out of bounds!")
    
if np.any((centers > 0.13) | (0.08 > centers)):
    raise ValueError("centers are out of bounds!")
    
if np.any((corners > 0.045) | (corners < 0.008)):
    raise ValueError("corners are out of bounds!")

# Test 2

In [ ]:
xx = [0.0, 1.0, 2.0]
yy = [100.0, 200.0, 300.0]

edge = [
    [10.0, 0.0, 10.0],
    [ 0.0, 0.0,  0.0],
    [10.0, 0.0, 10.0]
]

fig, vals = draw_example(xx, yy, edge)
nbshow()

## Compare 1/4ths

In [ ]:
# ---- Make sure quadrants are evenly sampled
quads, *_ = np.histogram2d(*vals, bins=(xx, yy))

tot = quads.sum()
# fraction in each quadrant
frac = quads / tot
print("frac = ", frac.flatten())
# expected poisson error
err = 1.0 / np.sqrt(tot / 4.0)
err *= 2.0   # padding
extr = 0.25 * np.array([1.0 - err, 1.0 + err])
print("exp = ", 1.0/4.0, " +- ", err, "==>", extr)
# Make sure each quadrant is roughly equal
if np.any((frac < extr[0]) | (frac > extr[1])):
    raise ValueError("Uneven distribution of points!")

## Compare 1/16ths

In [ ]:
edges = [np.sort(np.concatenate([kale.utils.midpoints(ee), ee])) for ee in [xx, yy]]
octs, *_ = np.histogram2d(*vals, bins=edges)

tot = octs.sum()
frac = octs / tot
ave = tot / octs.size
err = np.sqrt(ave)
extr = [ave - err, ave + err]
extr = np.array(extr) / tot
print(f"{ave=}, {err=}, {extr=}")

print("frac=\n", frac)
corners = np.ix_([0, 3], [0, 3])
corners = frac[corners].flatten()
print("corners=\n", corners)

centers = np.ix_([1, 2], [1, 2])
centers = frac[centers].flatten()
print("centers=\n", centers)

diags_h = np.ix_([1, 2], [0, 3])
diags_h = frac[diags_h]
diags_v = np.ix_([0, 3], [1, 2])
diags_v = frac[diags_v]
diags = np.concatenate([diags_h, diags_v]).flatten()
print("diags=\n", diags)

if not np.all(corners[:, np.newaxis] > diags[np.newaxis, :]):
    err = "corner values are not all larger than diagonal values!"
    raise ValueError(err)
    
if not np.all(diags[:, np.newaxis] > centers[np.newaxis, :]):
    err = "diagonal values are not all larger than center values!"
    raise ValueError(err)
    
if np.any((diags > 0.07) | (diags < 0.03)):
    raise ValueError("diagonals are out of bounds!")
    
test = (corners > 0.16) | (corners < 0.08)
if np.any(test):
    raise ValueError("corners are out of bounds!")
    
if np.any((centers > 0.035) | (centers < 0.008)):
    raise ValueError("centers are out of bounds!")

# Test 3

In [ ]:
xx = [0.0, 1.0, 2.0, 3.0]
yy = [100.0, 200.0, 300.0]

edge = [
    [10.0, 0.0, 10.0],
    [10.0, 0.0, 10.0],
    [10.0, 0.0, 10.0],
    [10.0, 0.0, 10.0],
]

fig, vals = draw_example(xx, yy, edge)
nbshow()

In [ ]:
hist, *_ = np.histogram2d(*vals, bins=(3, 3))
tot = hist.sum()
frac = hist / tot
ave = tot / hist.size
err = np.sqrt(ave)
print(f"{ave=} {err=}")
print("frac=\n", frac)

rows = np.mean(frac, axis=0)
print("rows=", rows)

df = np.fabs(rows[-1] - rows[0])
print("df=", df)
if df > 0.015:
    raise ValueError("Edge rows do not match!")

if (rows[0] < 0.12) or (rows[0] > 0.16):
    raise ValueError("edge rows are out of bounds!")
    
if (rows[1] < 0.03) or (rows[1] > 0.08):
    raise ValueError("middle row is out of bounds!")

    
cols = np.mean(frac, axis=1)
print("cols=", cols)
df = np.fabs(cols[:, np.newaxis] - cols[np.newaxis, :]).max()
print("max diff=", df)
if df > 0.02:
    raise ValueError("columns are inconsistent!")

if (cols[0] < 0.095) or (cols[0] > 0.125):
    raise ValueError("columns are out of bounds!")

# Visual : test 3D function

In [ ]:
def func(x, y, z):
    rv = (x + y - 2*z)**2 + x**2 + y**2 + z**2
    rv = 3 * np.exp(-rv / 2.0)
    return rv

xx = np.linspace(-4, 4, 200)
yy = np.linspace(-4, 4, 300)
zz = np.linspace(-1, 1, 5)

data = func(xx[:, np.newaxis, np.newaxis], yy[np.newaxis, :, np.newaxis], zz[np.newaxis, np.newaxis, :]) 
vals = kale.sample.sample_grid([xx, yy, zz], data, 2000)

# smap = zplot.smap(data, scale='linear')

# for ii, _ in enumerate(zz):
#     fig, ax = plt.subplots()
#     aa = data[:, :, ii]
#     plt.pcolormesh(xx, yy, aa.T, cmap=smap.cmap, norm=smap.norm)
    
# nbshow()
    

In [ ]:
# smap = zplot.smap(data_edge, scale='lin')
extr = kale.utils.minmax(data)
norm = mpl.colors.Normalize(*extr)

# add an offset so that the last 'bin' is also matched
off = np.diff(zz)[0]*0.5
keys = np.searchsorted(zz, vals[2] + off) - 1
for ii in range(zz.size):
    idx = (keys == ii)
        
    fig, ax = plt.subplots()
    plt.pcolormesh(xx, yy, data[:, :, ii].T, norm=norm, shading='auto')
    uu = vals[0][idx]
    vv = vals[1][idx]
    plt.scatter(uu, vv, facecolor='pink', edgecolor='0.25', alpha=0.7, s=20, zorder=100)

nbshow()

# Visual : test 3D random data

In [ ]:
data = kale.utils._random_data_3d_01()
data = data[:, ::10]

In [ ]:
kde = kale.KDE(data)
edges, values = kde.density()
print([len(ee) for ee in edges], np.shape(values))

In [ ]:
nsamp = 1000
samples = kale.sample.sample_grid(edges, values, nsamp)

In [ ]:
corner = kale.Corner(kde)
corner.plot_kde()
corner.plot_data(samples, color='r')

nbshow()

# Scalar Values

In [ ]:
def func(x, y):
    rv = (x + y)**2 + x**2 + y**2
    rv = 3 * np.exp(-rv / 2.0) + 1.0
    return rv

xx = np.linspace(-4, 4, 23)
yy = np.linspace(-4, 4, 25)

data = func(xx[:, np.newaxis], yy[np.newaxis, :]) 
vals = kale.sample.sample_grid([xx, yy], data, 100000)

In [ ]:
fig, axes = plt.subplots(figsize=[15, 6], ncols=2)

ax = axes[0]
ax.pcolormesh(xx, yy, data.T, shading='auto')

ax = axes[1]
hist, *_ = np.histogram2d(*vals, bins=(xx, yy))
ax.pcolormesh(xx, yy, hist.T, shading='auto')

nbshow()

In [ ]:
rr = xx[:, np.newaxis]**2 + yy[np.newaxis, :]**2 + 1.0
ww = np.sqrt(rr)
print(kale.utils.stats_str(ww))
# ww = np.ones_like(rr)
vv, scal = kale.sample.sample_grid_proportional([xx, yy], data, ww, 100000)

In [ ]:
fig, axes = plt.subplots(figsize=[15, 6], ncols=3)

ax = axes[0]
ax.pcolormesh(xx, yy, data.T, shading='auto')

ax = axes[1]
hist, *_ = np.histogram2d(*vv, bins=(xx, yy))
ax.pcolormesh(xx, yy, hist.T, shading='auto')

ax = axes[2]
hist, *_ = np.histogram2d(*vv, bins=(xx, yy), weights=scal)
ax.pcolormesh(xx, yy, hist.T, shading='auto')

nbshow()

# Outlier Sampling

## Weights and Normalizations

In [ ]:
def func_temp(xx):
    zz = np.power(xx, +1.5) * np.exp(-xx)
    return zz

In [ ]:
def get_outlier_stuff(NUM, threshold, poisson_inside):
    xx = kale.utils.spacing([1e-2, 1e1], scale='log', num=300)
    yy = func_temp(xx)
    Y = np.cumsum(yy)
    norm = NUM / Y[-1]
    yy *= norm
    Y *= norm
    dydx = np.diff(Y) / np.diff(xx)
    xc = kale.utils.midpoints(xx)
    nbins = xx.size - 1

    ss, ww = kale.sample_outliers(
        xc, dydx, threshold, poisson_inside=poisson_inside, # nsamp=NUM
    )
    print(f"{ww.size=}, {kale.utils.stats(ww)=}")
    wdist, _ = np.histogram(ss, bins=xx, weights=ww)

    return xx, yy, ss, xc, wdist

xx, yy, ss, xc, wdist = get_outlier_stuff(1e3, 10.0, False)

fig, ax = plt.subplots(figsize=[8, 3])
ax.set(xlim=[0.2, 11.0])
ax.set(xscale='log', yscale='log')
# ax.set(xlim=[0.2, 6.0], ylim=[3.0, 80.0])
kw = dict(alpha=0.75, lw=1.0)

ax.plot(xx, yy, 'k-', label='truth', **kw)
# ax.plot(xc, dist, 'r-', **kw)
ax.plot(xc, wdist, 'b-', label='samples', **kw)

tw = ax.twinx()
tw.set(ylim=[-0.2, 0.2])
yc = kale.utils.midpoints(yy)
diff = (wdist - yc) / yc
tw.plot(xc, diff, 'r-', label='difference')

ax.legend()
tw.legend()
nbshow()

## Example 1

In [ ]:
NTOT = 1e4

xx = np.logspace(-4, 4, 31)
yy = np.logspace(-2, 2, 21)
edges = (xx, yy)

xg, yg = np.meshgrid(*edges, indexing='ij')


aa = (xg/10 + (yg/10)**2) / np.power(xg * yg, 0.2)
pdf = np.power(aa, 2.0) / (1 + aa)**4
pdf = np.power(pdf, 0.25)

pdf *= NTOT / pdf.sum()

pmf = kale.utils.trapz_dens_to_mass(pdf, edges)
print(f"pdf = {kale.utils.stats_str(pdf)}")
print(f"pmf = {kale.utils.stats_str(pmf)}")

fig, ax = plt.subplots(figsize=[12, 8])
ax.set(xscale='log', yscale='log')
ax.pcolormesh(xg, yg, pdf, shading='nearest')

nbshow()

In [ ]:
thresh = 10.0
sampler = kale.Sample_Outliers(edges, pdf, threshold=thresh)

num, vv, ww = sampler.sample()
print(num, vv.size, ww.size, ww.sum(), pmf.sum(), ww.sum()/pmf.sum())

In [ ]:
fig, axes = plt.subplots(figsize=[20, 7], ncols=3)
for ax in axes:
    ax.set(xscale='log', yscale='log')
    
ax = axes[0]
extr = np.fabs(zz).max() * 1.1
norm = mpl.colors.Normalize(0.0, extr)

pcm = ax.pcolormesh(xg, yg, pmf, norm=norm)
plt.colorbar(pcm, ax=ax)

ax = axes[1]
hist, *_ = np.histogram2d(*vv, weights=ww, bins=(xx, yy))
xmid = kale.utils.midpoints(xx, log=True)
ymid = kale.utils.midpoints(yy, log=True)
pcm = ax.pcolormesh(xmid, ymid, hist.T, shading='nearest', norm=norm)
plt.colorbar(pcm, ax=ax)



ax = axes[2]
err = (hist - pmf) / pmf
extr = np.fabs(err).max()
norm = mpl.colors.Normalize(-extr, extr)
# norm = mpl.colors.Normalize(-0.1, 0.1)

pcm = ax.pcolormesh(xg, yg, err, cmap='bwr', norm=norm)
plt.colorbar(pcm, ax=ax)

nbshow()


idx = (pmf > thresh)
print(f"fraction above threshold: {np.count_nonzero(idx)/idx.size:.4e}")
check = err[idx]
print(f"errors above thresh     : {kale.utils.stats_str(check, format=':.4e')}")
if np.any(np.fabs(check) > 1e-10):
    raise ValueError(f"Error too large for interior region!")


idx = (pmf < thresh)
print(f"fraction below threshold: {np.count_nonzero(idx)/idx.size:.4e}")
check = np.fabs(hist - pmf)[idx]
allow = 6 * np.sqrt(np.maximum(pmf[idx], 1.0))
print(f"errors below thresh     : {kale.utils.stats_str(check, format=':.4e')}")
print(f" compared to 4x Poisson : {kale.utils.stats_str(check/allow, format=':.4e')}")
if np.any(check > allow):
    raise ValueError(f"Error too large for outlier region!")


## Example 2

In [ ]:
NTOT = 3e5

xx = np.logspace(-4, 4, 31)
yy = np.logspace(-2, 2, 21)

xg, yg = np.meshgrid(xx, yy, indexing='ij')

pdf = np.power(xg, -5.5) * np.power(yy, -1.35)
pdf = np.power(pdf, 0.25)

ledges = [np.log10(xx), np.log10(yy)]
pmf = kale.utils.trapz_dens_to_mass(pdf, ledges)
pdf *= NTOT / pmf.sum()
pmf = kale.utils.trapz_dens_to_mass(pdf, ledges)

print(f"pdf = {kale.utils.stats_str(pdf)}")
print(f"pmf = {kale.utils.stats_str(pmf)}")

fig, ax = plt.subplots(figsize=[16, 10])
ax.set(xscale='log', yscale='log')
pcm = ax.pcolormesh(xg, yg, np.log10(pdf), shading='nearest')
plt.colorbar(pcm, ax=ax)

nbshow()

In [ ]:
thresh = 10.0
sampler = kale.Sample_Outliers(ledges, pdf, threshold=thresh)

num, vv, ww = sampler.sample()
vv = np.power(10.0, vv)
print(num, vv.size, ww.size, ww.sum(), pmf.sum(), ww.sum()/pmf.sum())

In [ ]:
REALS = 100
sh = (xx.size - 1, yy.size - 1, REALS)
hist = np.zeros(sh)
for rr in range(REALS):
    nn, vv, ww = sampler.sample()
    vv = np.power(10.0, vv)
    hist[..., rr], *_ = np.histogram2d(*vv, weights=ww, bins=(xx, yy))
    


In [ ]:
def plot_sample_comparison(xx, yy, pdf, pmf, hist, log_flag=True, diff_extr=None):
    LOG_FLAG = True

    fig, axes = plt.subplots(figsize=[20, 7], ncols=3)
    for ax in axes:
        ax.set(xscale='log', yscale='log')

    def get_temp(vals):
        temp = vals
        if log_flag:
            _temp = np.zeros_like(temp)
            idx = (temp > 0.0)
            _temp[idx] = np.log10(temp[idx])
            temp = _temp
            del _temp
        return temp    

    temp = get_temp(pdf)
    extr = np.fabs(temp).max() * 1.1
    norm = mpl.colors.Normalize(0.0, extr)

    ax = axes[0]
    pcm = ax.pcolormesh(xg, yg, temp, shading='nearest', norm=norm)
    plt.colorbar(pcm, ax=ax)

    ax = axes[1]
    # hist, *_ = np.histogram2d(*vv, weights=ww, bins=(xx, yy))
#     xmid = kale.utils.midpoints(xx, log=True)
#     ymid = kale.utils.midpoints(yy, log=True)
    temp = get_temp(hist.T)
    pcm = ax.pcolormesh(xmid, ymid, temp, shading='nearest', norm=norm)
    plt.colorbar(pcm, ax=ax)

    ax = axes[2]
    zmid = kale.utils.midpoints(zz, log=False, axis=None)
    err = (hist - pmf) / pmf
    extr = np.fabs(err).max()
    if diff_extr is not None:
        extr = diff_extr

    norm = mpl.colors.Normalize(-extr, extr)

    pcm = ax.pcolormesh(xg, yg, err, cmap='bwr', norm=norm)
    plt.colorbar(pcm, ax=ax)
    return err
    
    
diff_extr = None
diff_extr = 10

err = plot_sample_comparison(xx, yy, pdf, pmf, hist.mean(axis=-1), log_flag=True, diff_extr=diff_extr)
nbshow()


idx = (pmf > thresh)
print(f"fraction above threshold: {np.count_nonzero(idx)/idx.size:.4e}")
check = err[idx]
print(f"errors above thresh     : {kale.utils.stats_str(check, format=':.4e')}")
if np.any(np.fabs(check) > 1e-10):
    raise ValueError(f"Error too large for interior region!")



idx = (pmf < thresh)
print(f"fraction below threshold: {np.count_nonzero(idx)/idx.size:.4e}")
check = np.fabs(hist.mean(axis=-1) - pmf)[idx]
allow = 4 * np.sqrt(np.maximum(pmf[idx], 1.0))
print(f"errors below thresh     : {kale.utils.stats_str(check, format=':.4e')}")
print(f" compared to 4x Poisson : {kale.utils.stats_str(check/allow, format=':.4e')}")
if np.any(check > allow):
    raise ValueError(f"Error too large for outlier region!")


## Example 3

In [ ]:
NTOT = 1e5

xx = np.logspace(-4, 4, 31)
yy = np.logspace(-2, 2, 21)
zz = np.linspace(0.0, 4.0, 11)

ledges = [np.log10(xx), np.log10(yy), zz]
xg, yg, zg = np.meshgrid(xx, yy, zz, indexing='ij')

aa = (xg/10 + (yg/10)**2) / np.power(xg * yg, 0.2)
pdf = np.power(aa, 2.0) / (1 + aa)**4
pdf = np.power(pdf, 0.25)

pmf = kale.utils.trapz_dens_to_mass(pdf, ledges)
pdf *= NTOT / pmf.sum()
pmf = kale.utils.trapz_dens_to_mass(pdf, ledges)

print(f"pdf = {kale.utils.stats_str(pdf)}")
print(f"pmf = {kale.utils.stats_str(pmf)}")


fig, axes = plt.subplots(figsize=[20, 7], ncols=3)
    
grids = [xg, yg, zg]
cut = tuple([slice(None), slice(None), 0])
for ii, ax in enumerate(axes):
    jj = (ii + 1) % 3
    kk = (ii + 2) % 3
    cc = [slice(None) for ii in range(3)]
    cc[ii] = 0
    cc = tuple(cc)
    aa = grids[jj]
    bb = grids[kk]
    
    xsc = 'linear' if jj == 2 else 'log'
    ysc = 'linear' if kk == 2 else 'log'
    ax.set(xscale=xsc, xlim=kale.utils.minmax(aa), yscale=ysc, ylim=kale.utils.minmax(bb))

    ax.pcolormesh(aa[cc], bb[cc], pmf[cc])

nbshow()

In [ ]:
thresh = 20.0
sampler = kale.Sample_Outliers(ledges, pdf, threshold=thresh)

num, vv, ww = sampler.sample()
vv[0] = np.power(10.0, vv[0])
vv[1] = np.power(10.0, vv[1])
print(num, vv.size, ww.size, ww.sum(), pmf.sum(), ww.sum()/pmf.sum())

In [ ]:
fig, axes = plt.subplots(figsize=[20, 7], ncols=3)
for ax in axes:
    ax.set(xscale='log', yscale='log')
    
dd = np.sum(pmf, axis=-1)
extr = np.fabs(dd).max() * 1.1
norm = mpl.colors.Normalize(0.0, extr)

ax = axes[0]
cut = tuple([slice(None), slice(None), 0])
pcm = ax.pcolormesh(xg[cut], yg[cut], dd, norm=norm)
plt.colorbar(pcm, ax=ax)

ax = axes[1]
hist, *_ = np.histogram2d(vv[0], vv[1], weights=ww, bins=(xx, yy))
# xmid = kale.utils.midpoints(xx, log=True)
# ymid = kale.utils.midpoints(yy, log=True)
pcm = ax.pcolormesh(xx, yy, hist.T, norm=norm)
plt.colorbar(pcm, ax=ax)

ax = axes[2]
# dd = kale.utils.midpoints(dd, log=True, axis=None)
err = (hist - dd) / dd
extr = np.fabs(err).max()
# extr = 0.3
# extr = 1.0
norm = mpl.colors.Normalize(-extr, extr)
# norm = mpl.colors.Normalize(-0.1, 0.1)

pcm = ax.pcolormesh(xg[cut], yg[cut], err, cmap='bwr', norm=norm)
plt.colorbar(pcm, ax=ax)

nbshow()


idx = np.all(pmf > thresh, axis=-1)
print(f"fraction above threshold: {np.count_nonzero(idx)/idx.size:.4e}")
check = err[idx]
print(f"errors above thresh     : {kale.utils.stats_str(check, format=':.4e')}")
if np.any(np.fabs(check) > 1e-10):
    raise ValueError(f"Error too large for interior region!")



idx = ~idx
# idx = slice(None)
# print(f"fraction below threshold: {np.count_nonzero(idx)/idx.size:.4e}")
check = np.fabs(hist - dd)[idx]
# allow = 3 * np.sqrt(np.maximum(dd[idx], 1.0))
allow = 2 * np.fabs(dd[idx] - sp.stats.poisson.ppf(1.0 - 1.0/check.size, dd[idx]))
print(f"errors below thresh     : {kale.utils.stats_str(check, format=':.4e')}")
print(f"    compared to allowed : {kale.utils.stats_str(check/allow, format=':.4e')}")
if np.any(check > allow):
    
    bad = np.where(check > allow)
    print(f"{bad=}, {hist[idx][bad]=}, {dd[idx][bad]=}")
    
    raise ValueError(f"Error too large for outlier region!")


## Example 4

In [ ]:
xx = np.logspace(-4, 4, 31)
yy = np.logspace(-2, 2, 21)
zz = np.linspace(0.0, 4.0, 11)
# zz = np.logspace(-3, 3, 11)

ledges = [np.log10(xx), np.log10(yy), zz]

xg, yg, zg = np.meshgrid(xx, yy, zz, indexing='ij')

pdf = 2.0 * np.ones_like(xg)
idx = ((1e-2 < xg) & (xg < 1e2)) & ((1e-1 < yg) & (yg < 1e1))

pdf[idx] = 30.0

pmf = kale.utils.trapz_dens_to_mass(pdf, ledges)
mid = tuple([ss.size//2 for ss in [xx, yy, zz]])
print(pdf[mid], pmf[mid])
pdf *= 30.0 / pmf[mid]
pmf = kale.utils.trapz_dens_to_mass(pdf, ledges)

print(f"pdf = {kale.utils.stats_str(pdf)}")
print(f"pmf = {kale.utils.stats_str(pmf)}")


fig, axes = plt.subplots(figsize=[20, 7], ncols=3)
# for ax in axes:
#     ax.set(xscale='log', yscale='log')
    
grids = [xg, yg, zg]
cut = tuple([slice(None), slice(None), 0])
for ii, ax in enumerate(axes):
    jj = (ii + 1) % 3
    kk = (ii + 2) % 3
    cc = [slice(None) for ii in range(3)]
    cc[ii] = ledges[ii].size//2
    cc = tuple(cc)
    aa = grids[jj]
    bb = grids[kk]

    xsc = 'linear' if jj == 2 else 'log'
    ysc = 'linear' if kk == 2 else 'log'
    
    pcm = ax.pcolormesh(aa[cc], bb[cc], pmf[cc])
    ax.set(xscale=xsc, xlim=kale.utils.minmax(aa), yscale=ysc, ylim=kale.utils.minmax(bb))
    plt.colorbar(pcm, ax=ax)

# ax = axes[0]
# ax.pcolormesh(xg[cut], yg[cut], dist[cut], shading='nearest')

nbshow()


In [ ]:
thresh = 10.0
sampler = kale.Sample_Outliers(ledges, pdf, threshold=thresh)

num, vv, ww = sampler.sample()
vv[0] = np.power(10.0, vv[0])
vv[1] = np.power(10.0, vv[1])
print(num, vv.shape, ww.size, ww.sum(), pmf.sum(), ww.sum()/pmf.sum())

In [ ]:
fig, axes = plt.subplots(figsize=[20, 7], ncols=3)
for ax in axes:
    ax.set(xscale='log', yscale='log')
    

ax = axes[0]
cut = tuple([slice(None), slice(None), 0])
dd = np.sum(pmf, axis=-1)

extr = np.fabs(dd).max() * 1.1
norm = mpl.colors.Normalize(0.0, extr)

pcm = ax.pcolormesh(xg[cut], yg[cut], dd, norm=norm)
plt.colorbar(pcm, ax=ax)

ax = axes[1]
hist, *_ = np.histogram2d(vv[0], vv[1], weights=ww, bins=(xx, yy))
xmid = kale.utils.midpoints(xx, log=True)
ymid = kale.utils.midpoints(yy, log=True)
pcm = ax.pcolormesh(xmid, ymid, hist.T, shading='nearest', norm=norm)
plt.colorbar(pcm, ax=ax)

ax = axes[2]
# dd = kale.utils.midpoints(dd, log=False, axis=None)
# dd /= (zz.size / (zz.size - 1))
err = (hist - dd) / dd
extr = np.fabs(err).max()
# extr = 0.01
# extr = 1.0
norm = mpl.colors.Normalize(-extr, extr)
# norm = mpl.colors.Normalize(-0.1, 0.1)

pcm = ax.pcolormesh(xg[cut], yg[cut], err, cmap='bwr', norm=norm)
plt.colorbar(pcm, ax=ax)

nbshow()


idx = np.all(pmf > thresh, axis=-1)
print(f"fraction above threshold: {np.count_nonzero(idx)/idx.size:.4e}")
check = err[idx]
print(f"errors above thresh     : {kale.utils.stats_str(check, format=':.4e')}")
if np.any(np.fabs(check) > 1e-10):
    raise ValueError(f"Error too large for interior region!")



idx = ~idx
print(f"fraction below threshold: {np.count_nonzero(idx)/idx.size:.4e}")
check = np.fabs(hist - dd)[idx]
# allow = 3 * np.sqrt(np.maximum(dd[idx], 1.0))
allow = 2 * np.fabs(dd[idx] - sp.stats.poisson.ppf(1.0 - 1.0/check.size, dd[idx]))
print(f"errors below thresh     : {kale.utils.stats_str(check, format=':.4e')}")
print(f"    compared to allowed : {kale.utils.stats_str(check/allow, format=':.4e')}")
if np.any(check > allow):
    raise ValueError(f"Error too large for outlier region!")


# holodeck error case

In [ ]:
# fname = "kalepy_sample_outliers_error.npz"
# fname = os.path.abspath(os.path.join("/Users/lzkelley/Desktop", fname))
# data = np.load(fname, allow_pickle=True)
# print(list(data.keys()))
# edges = data['edges']
# dens = data['dens']
# print(dens.shape)
# fig, ax = plt.subplots(figsize=[6, 3])
# mesh = np.meshgrid(*edges, indexing='ij')
# ax.pcolormesh(*mesh, np.log10(dens))

# for (ii, jj), dd in np.ndenumerate(dens):
#     x = edges[0][ii]
#     y = edges[1][jj]
#     ax.text(x, y, f"{dd:.0f}", transform=ax.transData, fontsize=6, color='r')

# nbshow()

In [ ]:
# vv, ww = kale.sample_outliers(edges, dens, 100.0)
# fig, axes = plt.subplots(figsize=[8, 4], ncols=2)
# for ii, ax in enumerate(axes):
#     ee = edges[ii]
#     idx = (ww == 1.0)
#     ax.hist(vv[ii], bins=ee, weights=ww, histtype='step', lw=2.0, alpha=0.5)
#     rr = kale.utils.subdivide(ee, 3)
#     ax.hist(vv[ii], bins=rr, weights=ww, histtype='step', lw=2.0, alpha=0.5)
    
# nbshow()

# fig, ax = plt.subplots(figsize=[8, 6])
# size = ww / (ww.max() - ww.min())
# print(kale.utils.stats_str(size))
# size = 40.0 * np.power(size, 1) + 1

# ax.scatter(*vv, s=size)
# nbshow()